# Astronomoical Source Detection Tutorial

Import numpy and sep as usual.

In [ ]:
import numpy as np
import sep

Importing the additional setup for reading the test image and displaying plots.

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
#Ellipse will be used later in the code for object detection
from matplotlib.patches import Ellipse
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

Adding a color class to change font colors and styles.

In [ ]:
#use colors.subclass(or command; e.g bold).colorname to print
#examples: print(colors.bold, colors.fg.blue, "this will be bold and blue")
#everything after this will have that format until the following command
#is given: print(colors.reset, "now, this text will be normal")

class colors:
    reset='\033[0m'          #reset all colors with colors.reset
    bold='\033[01m'
    underline='\033[04m'
    strikethrough='\033[09m'
    reverse='\033[07m'
    class fg:                #foreground subclass
        black='\033[30m'
        red='\033[31m'
        green='\033[32m'
        orange='\033[33m'
        blue='\033[34m'
        purple='\033[35m'
        cyan='\033[36m'
        lightgrey='\033[37m'
        darkgrey='\033[90m'
        lightred='\033[91m'
        lightgreen='\033[92m'
        yellow='\033[93m'
        lightblue='\033[94m'
        pink='\033[95m'
        lightcyan='\033[96m'
    class bg:                #background subclass
        black='\033[40m'
        red='\033[41m'
        green='\033[42m'
        orange='\033[43m'
        blue='\033[44m'
        purple='\033[45m'
        cyan='\033[46m'
        lightgrey='\033[47m'

In [ ]:
data = fits.getdata("image.fits")

#keep in mind to replace the directory prior to running on personal terminal
#(e.g: ""/Users/carlisavannah/Desktop/image.fits")

#use the following if an error arises in this cell:
#data = data.byteswap(inplace=True).newbyteorder()

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig("fitsimage.png")

Measuring a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)

Printing the data type and shape.

In [ ]:
print(type(data))
print(data.shape)

You may also mask pixels with the following code:

In [ ]:
bkg = sep.Background(data, bw=64, bh=64, fw=3, fh=3)

Getting a "global mean and noise of the image background.

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Evaluating background as 2D array with same size as original image, showing it, and saving it as .png

In [ ]:
bkg_image = bkg.back()

# alternate way to get background evaluation:
#bkg_image = np.array(bkg)

plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

#saving as .png
plt.savefig("backgroundsubtraction1.png")

Now, doing the same as above, but with background noise.

In [ ]:
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

#saving as .png
plt.savefig("backgroundsubtraction_noise.png")

Now, we subtract the background

In [ ]:
data_sub = data - bkg

#It is also possible to subtract the background using
#bkg.subfrom(data)

## Object Detection

Now that we have subtacted the background, we can begin object detection
We're setting the threshold to be a constant of 1.5σ where σ=bkg.globalrms

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

#printing the number of objects detected

len(objects)

Using objects['x'] and objects['y'], we can get the centroid coordinates of the objects. 

To do a visual check of where the objects are, we will over-plot the object coords with basic shape parameters.

To do this we will used the imported Ellipse from matplotlib.patches, plot background subtracted images(comment 1), plot an ellipse for each object(comment 2), and saving image as a PNG(comment 3).

In [ ]:
#1
fig, ax = plt.subplots()
m, s    = np.mean(data_sub), np.std(data_sub)
im      = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#2
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width  = 6*objects['a'][i],
                height = 6*objects['b'][i],
                angle  = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

#3
plt.savefig("objectdetection.png")

There are many fields for objects that give various information, below is an output of the available fields

In [ ]:
objects.dtype.names

## Aperture Photometry

Performing a simple circular aperture photometry with a 3 pixel radius at the locations of the objects.

flux, fluxerr, and flag are all 1D arrays with one entry per object; we will print the first 10 results as well.

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

#printing the first ten results
for i in range(10):
    print(colors.bold,colors.fg.purple,"object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]),colors.reset)

Printing the flattened data's type

In [ ]:
print(type(data.flatten()))

Showing the histogram of the data

In [ ]:
histogram = plt.hist(data.flatten(),range=[6600.,7200.],bins='auto')

Comments, color class, and slight modifications by rojenkin